In [2]:
import pandas as pd
import numpy as np
import torch
from PIL import Image

In [2]:
d = pd.read_csv('./data/input/train.csv')
d = d.loc[(d.label == 0) | (d.label == 1)]
d.reset_index(drop=True, inplace=True)
y = d.label
X = d.drop('label', axis=1).values
# X[X > 0] = 1


X = torch.tensor(X.reshape(-1, 28, 28))/255

In [10]:
d  = np.array([[1,3,2],
               [4,5,6]])
d.argmax(axis=1)

array([1, 2])

In [16]:
from collections import defaultdict
class Monitor:
    # batch: dict(loss = 0, acc = 0)

    def __init__(self):
        self.data = dict(loss = list(), acc = list(), bestacc = list(), 
                         bestepoch = list(), bestmodel = list())

    @staticmethod
    def calculate_batch_accuracy(y, y_pred):
        return (y == y_pred).sum()/len(y)

    def show_last(self, measure: str):
        return self.data[measure][-1]
    
    def track_history(self, measure: str):
        return {epoch: value for epoch, value in enumerate(self.data[measure])}
        
m = Monitor()
m.data['loss'].append(0.5)
m.data['loss'].append(0.4)
m.track_history('loss')

{0: 0.5, 1: 0.4}

In [4]:
from torch.utils.data import Dataset, DataLoader
from typing import Optional, Tuple, List

class MnistDataset(Dataset):    
    def __init__(self, X, y, digits=Optional[List[int]]):
        self.X = X
        self.y = y
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]
    

def prepare_data(X, y, batch_size):
    dataset = MnistDataset(X, y, digits=digits)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    return loader


In [5]:
prepare_data

<function __main__.prepare_data(X, y, batch_size)>

In [6]:
class ConvNet(torch.nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.conv1 = torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.fc1 = torch.nn.Linear(28*28*32, 1)

    def forward(self, x):
        x = torch.nn.functional.relu(self.conv1(x))
        x = x.view(-1, 28*28*32)
        x = torch.sigmoid(self.fc1(x))
        return x
    
# generate simple model 
model = ConvNet()
criterion = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# prepare data
batch_size = 100
# digits = False
train_loader = prepare_data(X, y, batch_size)

# train model
n_epochs = 5
for epoch in range(n_epochs):
    for i, (X_batch, y_batch) in enumerate(train_loader):
        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch.float().view(-1, 1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if i % 100 == 0:
            print(f'Epoch {epoch}, loss: {loss.item()}')
    

NameError: name 'digits' is not defined